In [1]:
import sys
sys.path.append("../../../")

In [2]:
import pandas as pd
from experiment.ipinyou.load import read_data
from experiment.ipinyou.agge2.train import train_estimators, train_lr_and_show_results
from experiment.measure import ProcessMeasure
import os

In [3]:
df = pd.read_json('sampling.json')

In [4]:

def _generate_space(generators):
    first_generator = generators[0]
    other_generators = generators[1:]

    if len(other_generators) == 0:
        for x in first_generator:
            yield x,
        return

    other = list(_generate_space(other_generators))
    for _first in first_generator:
        for _other in other:
            yield (_first,) + _other


def generate_space(generators, start=0, end=None):
    space = list(enumerate(list(_generate_space(generators))))
    if end is None:
        return space[start:]
    return space[start:end]


def neg_sample(df, ratio):
    clicks = df[df.click == 1]
    not_clicks = df[df.click == 0]
    return pd.concat([clicks, not_clicks.sample(int(df.shape[0] * ratio))], ignore_index=True)


if __name__ == '__main__':
    measure = ProcessMeasure()
    experiments = generate_space([
        # advertiser ids
        ['3476', '2259', '2261', '2821', '2997', '1458', '3358', '3386', '3427'],
#         ['3358', '3427'],
        # bins
        # [1, 5, 10, 50, 150, 300],
        [1, 10, 50, 150, 300],
        # re-runs
        # list(range(3)),
        list(range(30)),
        # bin type
        ['qcut'],
    ],
        # starting experiment id (you can skip start=N experiments in case of error)
        start=0)
    print(experiments)
    prev_subject = None
    df_train, df_test = (None, None)

    reuse = False

    for experiment_id, (subject, bins, attempt, bin_type) in experiments:

        if reuse and os.path.isfile('test.pckl') and os.path.isfile('train.pckl'):
            _df_train = pd.read_pickle('train.pckl')
            _df_test = pd.read_pickle('test.pckl')
        else:
            if subject != prev_subject:
                df_train, df_test = read_data(subject)
                df_train.drop(columns=['usertag'], inplace=True)
                df_test.drop(columns=['usertag'], inplace=True)

            prev_subject = subject

            test_sampling =  float(df[(df.advertiser == int(subject)) & (df.type == 'test')]['ctr']) * 2
            train_sampling = float(df[(df.advertiser == int(subject)) & (df.type == 'train')]['ctr']) * 2
            _df_test = neg_sample(df_test, test_sampling)#.sample(frac=0.1).reset_index()
            _df_train = neg_sample(df_train, train_sampling)#.sample(frac=0.1).reset_index()       
#             if subject in {'1458', '3386'}:
#                 _df_test = neg_sample(df_test, 0.0005)
#                 _df_train = neg_sample(df_train, 0.0005)
#             else:
#                 _df_test = neg_sample(df_test, 0.002)
#                 _df_train = neg_sample(df_train, 0.002)

#             _df_train.to_pickle('train.pckl')
#             _df_test.to_pickle('test.pckl')

        __df_train, __df_test, estimators = train_estimators(_df_train, _df_test, None,
                                                             normalize_column="minmax",
                                                             bins=bins,
                                                             bin_type=bin_type)
    
        # experiment_output_K_L_P_ES
        fname = 'experiment_v2_output_K=A_L=05a_P=1_ES=1.json'
        new_path = '05a'
        C=None
        norm_type = None
        measure.set_suffix('_None_f={}_b={}_bt={}_r=L2'.format(__df_train.shape[1], bins, bin_type))
        lr = train_lr_and_show_results(__df_train, __df_test, None, subject + '_' + str(C) + "_" + str(bins), measure=measure, C=C,
                                       norm_type=norm_type, cnt_vec=None, f_name=fname, new_path=new_path)
        print('Done experiment id={}, adv={}, bins={}, attempt={}'.format(experiment_id, subject, bins, attempt))

        for C in [2, 4, 6, 8]:
#             norm_type = 'L2'
#             measure.set_suffix('_None_f={}_b={}_bt={}_r=L2'.format(__df_train.shape[1], bins, bin_type))
#             lr = train_lr_and_show_results(__df_train, __df_test, None, subject + '_' + str(C) + '_L2_' + str(bins), measure=measure, C=C,
#                                            norm_type=norm_type, cnt_vec=None, f_name=fname, new_path=new_path)
#             print('Done experiment id={}, adv={}, bins={}, attempt={}'.format(experiment_id, subject, bins, attempt))

            norm_type = 'L2+'
            measure.set_suffix('_None_f={}_b={}_bt={}_r=L2+'.format(__df_train.shape[1], bins, bin_type))
            lr = train_lr_and_show_results(__df_train, __df_test, None, subject + '_' + str(C) + '_L2+_' + str(bins), measure=measure, C=C,
                                           norm_type=norm_type, cnt_vec=None, f_name=fname, new_path=new_path)
            print('Done experiment id={}, adv={}, bins={}, attempt={}'.format(experiment_id, subject, bins, attempt))
    print('-------------------------------- RESULT --------------------------------')
    measure.print()

[(0, ('3476', 1, 0, 'qcut')), (1, ('3476', 1, 1, 'qcut')), (2, ('3476', 1, 2, 'qcut')), (3, ('3476', 1, 3, 'qcut')), (4, ('3476', 1, 4, 'qcut')), (5, ('3476', 1, 5, 'qcut')), (6, ('3476', 1, 6, 'qcut')), (7, ('3476', 1, 7, 'qcut')), (8, ('3476', 1, 8, 'qcut')), (9, ('3476', 1, 9, 'qcut')), (10, ('3476', 1, 10, 'qcut')), (11, ('3476', 1, 11, 'qcut')), (12, ('3476', 1, 12, 'qcut')), (13, ('3476', 1, 13, 'qcut')), (14, ('3476', 1, 14, 'qcut')), (15, ('3476', 1, 15, 'qcut')), (16, ('3476', 1, 16, 'qcut')), (17, ('3476', 1, 17, 'qcut')), (18, ('3476', 1, 18, 'qcut')), (19, ('3476', 1, 19, 'qcut')), (20, ('3476', 1, 20, 'qcut')), (21, ('3476', 1, 21, 'qcut')), (22, ('3476', 1, 22, 'qcut')), (23, ('3476', 1, 23, 'qcut')), (24, ('3476', 1, 24, 'qcut')), (25, ('3476', 1, 25, 'qcut')), (26, ('3476', 1, 26, 'qcut')), (27, ('3476', 1, 27, 'qcut')), (28, ('3476', 1, 28, 'qcut')), (29, ('3476', 1, 29, 'qcut')), (30, ('3476', 10, 0, 'qcut')), (31, ('3476', 10, 1, 'qcut')), (32, ('3476', 10, 2, 'qcut'

loaded df with columns = Index(['click', 'weekday', 'hour', 'bidid', 'timestamp', 'logtype',
       'ipinyouid', 'useragent', 'IP', 'region', 'city', 'adexchange',
       'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'bidprice',
       'payprice', 'keypage', 'advertiser', 'usertag', 'nclick',
       'nconversation'],
      dtype='object')
loading /Users/rwiatr/phd/make-ipinyou-data/3476/train.log.txt ... 
loaded df with columns = Index(['click', 'weekday', 'hour', 'bidid', 'timestamp', 'logtype',
       'ipinyouid', 'useragent', 'IP', 'region', 'city', 'adexchange',
       'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'bidprice',
       'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')
na columns: []
new_path = 05a
Done experiment id=0, adv=3476, bins=1, attempt=0
new_path = 05a
Report: a=0.5, reg=2, min=1

Done experiment id=15, adv=3476, bins=1, attempt=15
new_path = 05a
Report: a=0.5, reg=4, min=2.0, max=6.0
Done experiment id=15, adv=3476, bins=1, attempt=15
new_path = 05a
Report: a=0.5, reg=6, min=3.0, max=9.0
Done experiment id=15, adv=3476, bins=1, attempt=15
new_path = 05a
Report: a=0.5, reg=8, min=4.0, max=12.0
Done experiment id=15, adv=3476, bins=1, attempt=15
new_path = 05a
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 05a
Report: a=0.5, reg=2, min=1.0, max=3.0
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 05a
Report: a=0.5, reg=4, min=2.0, max=6.0
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 05a
Report: a=0.5, reg=6, min=3.0, max=9.0
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 05a
Report: a=0.5, reg=8, min=4.0, max=12.0
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 05a
Done experiment id=17, adv=3476, bins=1, attempt=17
new_path = 05a
Report: a=0.5, reg=2, min=1.0, max=3.0
Done experiment 

new_path = 05a
Done experiment id=32, adv=3476, bins=10, attempt=2
new_path = 05a
Report: a=0.5, reg=2, min=1.0, max=3.0
Done experiment id=32, adv=3476, bins=10, attempt=2
new_path = 05a
Report: a=0.5, reg=4, min=2.0, max=6.0
Done experiment id=32, adv=3476, bins=10, attempt=2
new_path = 05a
Report: a=0.5, reg=6, min=3.0, max=9.0
Done experiment id=32, adv=3476, bins=10, attempt=2
new_path = 05a
Report: a=0.5, reg=8, min=4.0, max=12.0
Done experiment id=32, adv=3476, bins=10, attempt=2
new_path = 05a
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 05a
Report: a=0.5, reg=2, min=1.0, max=3.0
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 05a
Report: a=0.5, reg=4, min=2.0, max=6.0
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 05a
Report: a=0.5, reg=6, min=3.0, max=9.0
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 05a
Report: a=0.5, reg=8, min=4.0, max=12.0
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 05a
D

Done experiment id=48, adv=3476, bins=10, attempt=18
new_path = 05a
Report: a=0.5, reg=8, min=4.0, max=12.0
Done experiment id=48, adv=3476, bins=10, attempt=18
new_path = 05a
Done experiment id=49, adv=3476, bins=10, attempt=19
new_path = 05a
Report: a=0.5, reg=2, min=1.0, max=3.0
Done experiment id=49, adv=3476, bins=10, attempt=19
new_path = 05a
Report: a=0.5, reg=4, min=2.0, max=6.0
Done experiment id=49, adv=3476, bins=10, attempt=19
new_path = 05a
Report: a=0.5, reg=6, min=3.0, max=9.0
Done experiment id=49, adv=3476, bins=10, attempt=19
new_path = 05a
Report: a=0.5, reg=8, min=4.0, max=12.0
Done experiment id=49, adv=3476, bins=10, attempt=19
new_path = 05a
Done experiment id=50, adv=3476, bins=10, attempt=20
new_path = 05a
Report: a=0.5, reg=2, min=1.0, max=3.0
Done experiment id=50, adv=3476, bins=10, attempt=20
new_path = 05a
Report: a=0.5, reg=4, min=2.0, max=6.0
Done experiment id=50, adv=3476, bins=10, attempt=20
new_path = 05a
Report: a=0.5, reg=6, min=3.0, max=9.0
Done e

...

Done experiment id=1346, adv=3427, bins=300, attempt=26
new_path = 05a
Report: a=0.5, reg=6, min=3.0, max=9.0
Done experiment id=1346, adv=3427, bins=300, attempt=26
new_path = 05a
Report: a=0.5, reg=8, min=4.0, max=12.0
Done experiment id=1346, adv=3427, bins=300, attempt=26
new_path = 05a
Done experiment id=1347, adv=3427, bins=300, attempt=27
new_path = 05a
Report: a=0.5, reg=2, min=1.0, max=3.0
Done experiment id=1347, adv=3427, bins=300, attempt=27
new_path = 05a
Report: a=0.5, reg=4, min=2.0, max=6.0
Done experiment id=1347, adv=3427, bins=300, attempt=27
new_path = 05a
Report: a=0.5, reg=6, min=3.0, max=9.0
Done experiment id=1347, adv=3427, bins=300, attempt=27
new_path = 05a
Report: a=0.5, reg=8, min=4.0, max=12.0
Done experiment id=1347, adv=3427, bins=300, attempt=27
new_path = 05a
Done experiment id=1348, adv=3427, bins=300, attempt=28
new_path = 05a
Report: a=0.5, reg=2, min=1.0, max=3.0
Done experiment id=1348, adv=3427, bins=300, attempt=28
new_path = 05a
Report: a=0.5, 

	"auc_2261_4_L2+_50_None_f=134_b=50_bt=qcut_r=L2+": [63.89615939319482],
	"delta_2261_6_L2+_50_None_f=134_b=50_bt=qcut_r=L2+": [0.32848405838012695],
	"auc_2261_6_L2+_50_None_f=134_b=50_bt=qcut_r=L2+": [64.36621975321832],
	"delta_2261_8_L2+_50_None_f=134_b=50_bt=qcut_r=L2+": [0.5395240783691406],
	"auc_2261_8_L2+_50_None_f=134_b=50_bt=qcut_r=L2+": [64.29677901821485],
	"delta_2261_None_50_None_f=144_b=50_bt=qcut_r=L2": [1.15736985206604],
	"auc_2261_None_50_None_f=144_b=50_bt=qcut_r=L2": [60.784680305539226],
	"delta_2261_2_L2+_50_None_f=144_b=50_bt=qcut_r=L2+": [0.4578511714935303],
	"auc_2261_2_L2+_50_None_f=144_b=50_bt=qcut_r=L2+": [60.81672987554083],
	"delta_2261_4_L2+_50_None_f=144_b=50_bt=qcut_r=L2+": [0.39888978004455566],
	"auc_2261_4_L2+_50_None_f=144_b=50_bt=qcut_r=L2+": [60.84343785054218],
	"delta_2261_6_L2+_50_None_f=144_b=50_bt=qcut_r=L2+": [0.40795302391052246],
	"auc_2261_6_L2+_50_None_f=144_b=50_bt=qcut_r=L2+": [60.816729875540844],
	"delta_2261_8_L2+_50_None_f=144_b

	"delta_2997_4_L2+_50_None_f=151_b=50_bt=qcut_r=L2+": [3.164370059967041, 3.4266817569732666],
	"auc_2997_4_L2+_50_None_f=151_b=50_bt=qcut_r=L2+": [52.112174176421334, 53.11071320398382],
	"delta_2997_6_L2+_50_None_f=151_b=50_bt=qcut_r=L2+": [2.984685182571411, 3.386892795562744],
	"auc_2997_6_L2+_50_None_f=151_b=50_bt=qcut_r=L2+": [51.86861938104656, 52.82226023126124],
	"delta_2997_8_L2+_50_None_f=151_b=50_bt=qcut_r=L2+": [3.367086172103882, 4.169221878051758],
	"auc_2997_8_L2+_50_None_f=151_b=50_bt=qcut_r=L2+": [51.57553405669286, 52.28187859675379],
	"delta_2997_None_50_None_f=148_b=50_bt=qcut_r=L2": [4.365969896316528, 5.488811016082764, 5.464018106460571],
	"auc_2997_None_50_None_f=148_b=50_bt=qcut_r=L2": [52.88426247617012, 52.8136191137955, 53.6935877563383],
	"delta_2997_2_L2+_50_None_f=148_b=50_bt=qcut_r=L2+": [3.142961263656616, 3.243396043777466, 4.2341392040252686],
	"auc_2997_2_L2+_50_None_f=148_b=50_bt=qcut_r=L2+": [54.77782528907655, 52.79722771571615, 52.94457213105992

	"delta_3358_4_L2+_50_None_f=178_b=50_bt=qcut_r=L2+": [4.805865049362183, 5.375578880310059],
	"auc_3358_4_L2+_50_None_f=178_b=50_bt=qcut_r=L2+": [73.46116792648584, 73.31888246628131],
	"delta_3358_6_L2+_50_None_f=178_b=50_bt=qcut_r=L2+": [4.418747901916504, 6.786297798156738],
	"auc_3358_6_L2+_50_None_f=178_b=50_bt=qcut_r=L2+": [73.30850748480806, 73.51600711427302],
	"delta_3358_8_L2+_50_None_f=178_b=50_bt=qcut_r=L2+": [4.560235023498535, 5.640661954879761],
	"auc_3358_8_L2+_50_None_f=178_b=50_bt=qcut_r=L2+": [73.64124796205721, 74.0191937157255],
	"delta_3358_None_150_None_f=182_b=150_bt=qcut_r=L2": [9.287084817886353, 7.457716941833496, 7.67126989364624, 12.34246826171875],
	"auc_3358_None_150_None_f=182_b=150_bt=qcut_r=L2": [70.04038832073513, 68.88098414109974, 71.4310063732029, 68.43967689343411],
	"delta_3358_2_L2+_150_None_f=182_b=150_bt=qcut_r=L2+": [6.450416326522827, 6.257936000823975, 6.4125587940216064, 9.569062948226929],
	"auc_3358_2_L2+_150_None_f=182_b=150_bt=qcut_r=

	"auc_3427_2_L2+_150_None_f=206_b=150_bt=qcut_r=L2+": [70.56805184902784, 69.10456519626531, 68.71285685452222],
	"delta_3427_4_L2+_150_None_f=206_b=150_bt=qcut_r=L2+": [4.894643068313599, 7.252280950546265, 7.366636037826538],
	"auc_3427_4_L2+_150_None_f=206_b=150_bt=qcut_r=L2+": [70.86669208285679, 69.06980481861063, 69.09409970621874],
	"delta_3427_6_L2+_150_None_f=206_b=150_bt=qcut_r=L2+": [5.7831130027771, 6.057025194168091, 5.918581962585449],
	"auc_3427_6_L2+_150_None_f=206_b=150_bt=qcut_r=L2+": [70.42863657090743, 68.98944480575302, 69.30565211216016],
	"delta_3427_8_L2+_150_None_f=206_b=150_bt=qcut_r=L2+": [6.502302885055542, 7.440959930419922, 7.317186594009399],
	"auc_3427_8_L2+_150_None_f=206_b=150_bt=qcut_r=L2+": [70.47423620611035, 68.85899994767254, 69.07167365611893],
	"delta_3427_None_150_None_f=205_b=150_bt=qcut_r=L2": [10.026273965835571, 11.361735105514526],
	"auc_3427_None_150_None_f=205_b=150_bt=qcut_r=L2": [71.2189679531744, 70.0225381803503],
	"delta_3427_2_L2+_